In [2]:
import pandas as pd
import numpy as np
from keras.utils import *

X_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", header=None, sep="\s+")
y_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", header=None)

X_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", header=None, sep="\s+")
y_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7352, 561, 1), (7352, 7), (2947, 561, 1), (2947, 7))

In [3]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *

def build_model(hp):
    modelo = Sequential()
    modelo.add(Bidirectional(LSTM(hp.Int("units_lstm_1", min_value=1, max_value=600), return_sequences=True)))
    modelo.add(Dropout(hp.Float("dropout_1", min_value=0.001, max_value=0.5)))
    modelo.add(Bidirectional(LSTM(hp.Int("units_lstm_2", min_value=1, max_value=600))))
    modelo.add(Dropout(hp.Float("dropout_2", min_value=0.001, max_value=0.5)))
    modelo.add(Dense(7, activation="sigmoid"))
    modelo.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.0001, 0.001, 0.01])), metrics=["accuracy"])
    return modelo

In [4]:
from keras_tuner import *
from keras.callbacks import *

tuner = BayesianOptimization(hypermodel=build_model, objective='accuracy')
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=10, callbacks=[EarlyStopping(patience=2)])

Trial 10 Complete [02h 28m 25s]
accuracy: 0.7754352688789368

Best accuracy So Far: 0.9344395995140076
Total elapsed time: 2d 16h 45m 20s


In [5]:
hyperparameters = tuner.get_best_hyperparameters()[0]
print(f"Units LSTM 1: {hyperparameters.get('units_lstm_1')}")
print(f"Dropout 1: {hyperparameters.get('dropout_1')}")
print(f"Units LSTM 2: {hyperparameters.get('units_lstm_2')}")
print(f"Dropout 2: {hyperparameters.get('dropout_2')}")
print(f"Learning Rate: {hyperparameters.get('learning_rate')}")

Units LSTM 1: 355
Dropout 1: 0.2591624128052411
Units LSTM 2: 347
Dropout 2: 0.36323394502080436
Learning Rate: 0.001


In [6]:
best_model = tuner.get_best_models()[0]
best_model.summary()

C:\Users\daniel.a.goncalves\Desktop\tedi\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 561, 710)            │       1,013,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 561, 710)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 694)                 │       2,937,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 694)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 7)                   │           4,865 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,955,753 (15.09 MB)

 Trainable params: 3,955,753 (15.09 MB)

 Non-trainable params: 0 (0.00 B)